In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from scipy.stats import gaussian_kde
from scipy.stats import norm
from scipy import integrate
from sklearn.neighbors import KernelDensity
import scipy.stats

np.random.seed(20200619)

lab_ranges = {'ALBUMIN':        [3.5, 5.5],                
              'BICARBONATE':    [21,29],                     
              'BUN':            [10,20],                                          
              'CALCIUM':        [8.5,10.5],                                       
              'FREECALCIUM':    [1.05,1.37],                  
              'CHLORIDE':       [98,106],                       
              'MAGNESIUM':      [1.8,3],                   
              'PHOSPHATE':      [3,4.5],                     
              'HEMOGLOBIN':     [12,18],                   
              'SODIUM':         [136,145],                
              'CREATININE':     [0.5,1.5],                   
              'PLATELET':       [150,400],                  
              'POTASSIUM':      [3.3,5.5],                 
              'LACTATE':        [0.5,2.0],                      
              'WBC':            [4.5,11],                     
              'GLUCOSE':        [75.0, 115.0]            
             } 

lab_units = {  'ALBUMIN':        'g/dL',          
              'BICARBONATE':    'mEq/L',           
              'BUN':            'mg/dL',                           
              'CALCIUM':        'mg/dL',                                
              'FREECALCIUM':    'mmol/L',        
              'CHLORIDE':       'mEq/L',             
              'MAGNESIUM':      'mg/dL',         
              'PHOSPHATE':      'mg/dL',           
              'HEMOGLOBIN':     'g/dL',          
              'SODIUM':         'mEq/L',        
              'CREATININE':     'mg/dL',           
              'PLATELET':       'K/uL',            
              'POTASSIUM':      'mEq/L',         
              'LACTATE':        'mmol/L',            
              'WBC':            'K/uL',            
              'GLUCOSE':        'mg/dL'         
             } 
lab_unit = {  #'ALBUMIN':        'g/dL',          
              #'BICARBONATE':    'mEq/L',           
              #'BUN':            'mg/dL',                           
              #'CALCIUM':        'mg/dL',                                
              #'FREECALCIUM':    'mmol/L',        
              'CHLORIDE':       'mEq/L',             
              #'MAGNESIUM':      'mg/dL',         
              #'PHOSPHATE':      'mg/dL',           
              #'HEMOGLOBIN':     'g/dL',          
              'SODIUM':         'mEq/L',        
              #'CREATININE':     'mg/dL',           
              #'PLATELET':       'K/uL',            
              #'POTASSIUM':      'mEq/L',         
              #'LACTATE':        'mmol/L',            
              #'WBC':            'K/uL',            
              #'GLUCOSE':        'mg/dL'         
             } 


data = pd.read_csv("S:/NUS/Year Two/UROPS/eicu_labminmax.csv")

mort_data = data[data['mort_icu']==1]
surv_data = data[data['mort_icu']==0]


In [6]:
def ResumeNorm(val1, val2):
    norm_val1 = norm.ppf(0.025)
    norm_val2 = norm.ppf(0.975)
    mean = (val2 - val1)/2 + val1
    scale = (val1 - val2) / (norm_val1 - norm_val2)
    return norm(mean, scale)

def OverlapParam(dist1, dist2, lab):
    """
        Overlapping coefficient: integration of min value of two distribution over R. 
        For chloride, glucose, sodium, use large interval with focus point interval instead
        
        return integrate.quad(func, -np.inf , np.inf). 
    """
    func = lambda x: min(dist1.pdf(x), dist2.pdf(x))
    if lab in ['CHLORIDE', 'SODIUM']:
        return integrate.quad(func, 0, 1000)
    else:
        return integrate.quad(func, -np.inf, np.inf)

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

In [3]:
c = ['labName','Best group Mean [IQR]','Worst group Mean [IQR]','P-Value']
df=pd.DataFrame(columns = c)
    
for i, u in iter(lab_unit.items()):
    
   

    # best vs worst
    
    n_surv = surv_data[i.lower() + '_min'].dropna().count()
    quarter = n_surv /4
    quarter = int(quarter)
    cond = data.mort_icu == 0
    all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna()
    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().head(quarter)
    worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().tail(quarter)
    
    [min_q1, min_q2] = best_group.quantile(q = [0.25,0.75])
    min_bgm = best_group.mean()
    [min_p1, min_p2] = worst_group.quantile(q = [0.25,0.75])
    min_wgm = worst_group.mean()


    t_stat, p_val_min = mannwhitneyu(best_group, worst_group)
    
    df=df.append(pd.DataFrame([['{}_min:'.format(i), 
                                '{} , [{}-{}]'.format(format(min_bgm, '.2f'), min_q1, min_q2), 
                                '{} , [{}-{}]'.format(format(min_wgm, '.2f'), min_p1, min_p2), 
                                p_val_min]], columns = c))
    
    
    n_surv = surv_data[i.lower() + '_max'].dropna().count()
    n_all = data[i.lower() + '_max'].dropna().count()
    quarter = n_surv /4
    quarter = int(quarter)
    cond = data.mort_icu == 0
    all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna()
    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna().head(quarter)
    worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna().tail(quarter)
    
    [max_q1, max_q2] = best_group.quantile(q = [0.25,0.75])
    max_bgm = best_group.mean()
    [max_p1, max_p2] = worst_group.quantile(q = [0.25,0.75])
    max_wgm = worst_group.mean()

    t_stat, p_val_max = mannwhitneyu(best_group, worst_group)
    
    df=df.append(pd.DataFrame([['{}_max:'.format(i), 
                                '{} , [{}-{}]'.format(format(max_bgm, '.2f'), max_q1, max_q2), 
                                '{} , [{}-{}]'.format(format(max_wgm, '.2f'), max_p1, max_p2), 
                                round(p_val_max,2)]], columns = c))
print(df)
                                

    

    
        

            labName     Best group Mean [IQR]    Worst group Mean [IQR]  \
0      ALBUMIN_min:          3.29 , [2.8-3.8]          2.90 , [2.4-3.4]   
0      ALBUMIN_max:          3.45 , [3.0-3.9]          3.14 , [2.6-3.7]   
0  BICARBONATE_min:       23.52 , [21.0-26.0]       22.47 , [19.0-25.2]   
0  BICARBONATE_max:       25.46 , [23.0-28.0]       25.52 , [22.0-28.0]   
0          BUN_min:       20.35 , [11.0-23.0]       25.18 , [12.0-31.0]   
0          BUN_max:       23.47 , [12.0-27.0]       30.51 , [15.0-38.0]   
0      CALCIUM_min:          8.42 , [8.0-8.9]          8.03 , [7.5-8.6]   
0      CALCIUM_max:          8.84 , [8.4-9.3]          8.68 , [8.2-9.2]   
0  FREECALCIUM_min:   1.11 , [1.06786-1.1976]  1.08 , [1.01796-1.17265]   
0  FREECALCIUM_max:  1.19 , [1.11776-1.22754]   1.16 , [1.0978-1.22255]   
0     CHLORIDE_min:     101.95 , [99.0-106.0]     101.51 , [98.0-106.0]   
0     CHLORIDE_max:    104.80 , [102.0-108.0]    105.80 , [102.0-110.0]   
0    MAGNESIUM_min:      

In [7]:
c = ['labName','OVL','SMD','P-Value']

#all vs normal
df=pd.DataFrame(columns = c)
    
for i, u in iter(lab_unit.items()):
 
    all_patients = data.sort_values(by=['los'])[i.lower()+'_min'].dropna()
    n_all = data[i.lower() + '_min'].dropna().count()
    
    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size=n_all, random_state=20200704)

    t_stat, p_val_min = mannwhitneyu(all_patients, norm_sample)
    
    #standardized mean difference
    smd_min = cohen_d(all_patients, norm_sample)
        
    #overlapping parameter
    all_patients_dist = gaussian_kde(all_patients)
    ovl_min = OverlapParam(all_patients_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_min:'.format(i), 
                                round(max(ovl_min),2), 
                                round(smd_min,2), 
                                round(p_val_min,2)]], columns = c))
    
    

    all_patients = data.sort_values(by=['los'])[i.lower()+'_max'].dropna()
    
    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size=n_all, random_state=20200704)

    t_stat, p_val_max = mannwhitneyu(all_patients, norm_sample)
        #standardized mean difference
    smd_max = cohen_d(all_patients, norm_sample)
        
    #overlapping parameter
    all_patients_dist = gaussian_kde(all_patients)
    ovl_max = OverlapParam(all_patients_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_max:'.format(i), 
                                round(max(ovl_max),2), 
                                round(smd_max,2),
                                round(p_val_max,2)]], columns = c))

print(df)
    
    

    

         labName   OVL   SMD  P-Value
0  CHLORIDE_min:  0.55 -0.04      0.0
0  CHLORIDE_max:  0.45  0.74      0.0
0    SODIUM_min:  0.54 -0.95      0.0
0    SODIUM_max:  0.73 -0.26      0.0


In [9]:
# best vs normal
c = ['labName','OVL','SMD','P-Value']

df=pd.DataFrame(columns = c)
    
for i, u in iter(lab_unit.items()):
 
    n_surv = surv_data[i.lower() + '_min'].dropna().count()
    quarter = n_surv /4
    quarter = int(quarter)
    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().head(quarter)
  
    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size = quarter,random_state=20200620)

    t_stat, p_val_min = mannwhitneyu(best_group, norm_sample)
    
    #standardized mean difference
    smd_min = cohen_d(best_group, norm_sample)
        
    #overlapping parameter
    best_group_dist = gaussian_kde(best_group)
    ovl_min = OverlapParam(best_group_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_min:'.format(i), 
                                round(max(ovl_min),2), 
                                round(smd_min,2), 
                                round(p_val_min,2)]], columns = c))
    
    n_surv = surv_data[i.lower() + '_max'].dropna().count()
    quarter = n_surv /4
    quarter = int(quarter)
    cond = data.mort_icu == 0
    all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna()
    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna().head(quarter)
    

    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size = quarter, random_state=20100704)

    t_stat, p_val_max = mannwhitneyu(best_group, norm_sample)
        #standardized mean difference
    smd_max = cohen_d(best_group, norm_sample)
        
    #overlapping parameter
    best_group_dist = gaussian_kde(best_group)
    ovl_max = OverlapParam(best_group_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_max:'.format(i), 
                                round(max(ovl_max),2), 
                                round(smd_max,2),
                                round(p_val_max,2)]], columns = c))

print(df)
    
    

         labName   OVL   SMD  P-Value
0  CHLORIDE_min:  0.57 -0.01      0.0
0  CHLORIDE_max:  0.49  0.65      0.0
0    SODIUM_min:  0.57 -0.94      0.0
0    SODIUM_max:  0.74 -0.42      0.0


In [10]:
# worst vs normal 
c = ['labName','OVL','SMD','P-Value']

df=pd.DataFrame(columns = c)
    
for i, u in iter(lab_unit.items()):

    worst_group = mort_data.sort_values(by=['los'])[i.lower()+'_min'].dropna()
    n_mort = mort_data[i.lower() + '_min'].dropna().count()

    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size=n_mort, random_state=20200704)

    t_stat, p_val_min = mannwhitneyu(worst_group, norm_sample)
    
    #standardized mean difference
    smd_min = cohen_d(worst_group, norm_sample)
        
    #overlapping parameter
    worst_group_dist = gaussian_kde(worst_group)
    ovl_min = OverlapParam(worst_group_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_min:'.format(i), 
                                round(max(ovl_min),2), 
                                round(smd_min,2), 
                                round(p_val_min,2)]], columns = c))
    
    
    worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_max'].dropna()
    n_mort = data[i.lower() + '_max'].dropna().count()
    
    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size=n_mort, random_state=20200704)

    t_stat, p_val_max = mannwhitneyu(worst_group, norm_sample)
        #standardized mean difference
    smd_max = cohen_d(worst_group, norm_sample)
        
    #overlapping parameter
    worst_group_dist = gaussian_kde(worst_group)
    ovl_max = OverlapParam(worst_group_dist, norm_dist,i)
    
    df=df.append(pd.DataFrame([['{}_max:'.format(i), 
                                round(max(ovl_max),2), 
                                round(smd_max,2),
                                round(p_val_max,2)]], columns = c))

print(df)
    
    



         labName   OVL   SMD  P-Value
0  CHLORIDE_min:  0.48 -0.15      0.0
0  CHLORIDE_max:  0.45  0.75      0.0
0    SODIUM_min:  0.49 -0.88      0.0
0    SODIUM_max:  0.74 -0.28      0.0


In [63]:
#plot best out come dist

i='ALBUMIN'
n_all = data[i.lower() + '_min'].dropna().count()
n_surv = surv_data[i.lower() + '_min'].dropna().count()
quarter = n_surv /4
quarter = int(quarter)
cond = data.mort_icu == 0
all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna()
best_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().head(quarter)
worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().tail(quarter)
    
[q1, q2] = all_patients.quantile(q = [0.25,0.75])

wgm = worst_group.mean()

t_stat, p_val = mannwhitneyu(best_group, worst_group)
    
norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

norm_sample = norm_dist.rvs(size=n_all, random_state=20171024)

smd_min = cohen_d(all_patients, norm_sample)
        
all_patients_dist = gaussian_kde(all_patients)
ovl_min = OverlapParam(all_patients_dist, norm_dist)
    
print(round(max(ovl_min),2))

# plot normal distribution with normal range
'''
    #plot best out come dist
    n_all = data[i.lower() + '_min'].dropna().count()
    n_surv = surv_data[i.lower() + '_min'].dropna().count()
    quarter = n_surv /4
    quarter = int(quarter)
    cond = data.mort_icu == 0
    all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna()
    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().head(quarter)
    worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().tail(quarter)
    
    [q1, q2] = all_patients.quantile(q = [0.25,0.75])
    allm = all_patients.mean()

    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])

    norm_sample = norm_dist.rvs(size=n_all, random_state=20171024)

    t_stat, p_val = mannwhitneyu(norm_sample, all_patients)
    print('{}_minimun:'.format)
    print('All patient: {} , [{}-{}]    {}'.format(format(allm, '.2f'), q1, q2, p_val))
'''

        

0.26


"\n    #plot best out come dist\n    n_all = data[i.lower() + '_min'].dropna().count()\n    n_surv = surv_data[i.lower() + '_min'].dropna().count()\n    quarter = n_surv /4\n    quarter = int(quarter)\n    cond = data.mort_icu == 0\n    all_patients = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna()\n    best_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().head(quarter)\n    worst_group = surv_data.sort_values(by=['los'])[i.lower()+'_min'].dropna().tail(quarter)\n    \n    [q1, q2] = all_patients.quantile(q = [0.25,0.75])\n    allm = all_patients.mean()\n\n    norm_dist = ResumeNorm(lab_ranges[i][0], lab_ranges[i][1])\n\n    norm_sample = norm_dist.rvs(size=n_all, random_state=20171024)\n\n    t_stat, p_val = mannwhitneyu(norm_sample, all_patients)\n    print('{}_minimun:'.format)\n    print('All patient: {} , [{}-{}]    {}'.format(format(allm, '.2f'), q1, q2, p_val))\n"